In [1]:
import pandas as pd
from collections import defaultdict, Counter

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# Baseball Dataset
Note: This data was downloaded from https://www.kaggle.com/pschale/mlb-pitch-data-20152018

Pitch-level data for every pitch thrown during the 2015-2018 MLB regular seasons. Data scraped from http://gd2.mlb.com/components/game/mlb/. Each row represents a single pitch.

The data doesn't come with clear definitions (that I can find, at least). Here's what I believe the codes mean:

## Pitch Type Definitions
CH - Changeup

CU - Curveball

EP - Eephus*

FC - Cutter

FF - Four-seam Fastball

FO - Pitchout (also PO)*

FS - Splitter

FT - Two-seam Fastball

IN - Intentional ball

KC - Knuckle curve

KN - Knuckeball

PO - Pitchout (also FO)*

SC - Screwball*

SI - Sinker

SL - Slider

UN - Unknown*

*these pitch types occur rarely

## Code Definitions
While these aren't spelled out anywhere, play descriptions allowed confident identification of these codes

B - Ball

*B - Ball in dirt

S - Swinging Strike

C - Called Strike

F - Foul

T - Foul Tip

L - Foul Bunt

I - Intentional Ball

W - Swinging Strike (Blocked)

M - Missed Bunt

P - Pitchout

Q - Swinging pitchout

R - Foul pitchout

Values that only occur on last pitch of at-bat:

X - In play, out(s)

D - In play, no out

E - In play, runs

H - Hit by pitch

Note: all codes, except for H, come directly from the XML files. All at-bats with code H were given no code in the XMLs.

In [2]:
# If you haven't already, uncomment and execute the following line to unzip the data
# ! unzip 90666_1164824_bundle_archive.zip

## Get and clean up pitches data

In [3]:
pitches = pd.read_csv('pitches.csv')
pitches.head(10)

,px,pz,start_speed,end_speed,spin_rate,spin_dir,break_angle,break_length,break_y,ax,ay,az,sz_bot,sz_top,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,pfx_x,pfx_z,nasty,zone,code,type,pitch_type,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,0.416,2.963,92.9,84.1,2305.052,159.235,-25.0,3.2,23.7,7.665,34.685,-11.960,1.72,3.56,2.000,-6.409,-136.065,-3.995,101.140000,2.280,158.780000,50.0,5.302,4.16,10.93,55.0,3.0,C,S,FF,3,0.0,2.015000e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.191,2.347,92.8,84.1,2689.935,151.402,-40.7,3.4,23.7,12.043,34.225,-10.085,1.72,3.56,2.000,-8.411,-135.690,-5.980,124.280000,2.119,175.410000,50.0,5.307,6.57,12.00,31.0,5.0,S,S,FF,4,0.0,2.015000e+09,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,-0.518,3.284,94.1,85.2,2647.972,145.125,-43.7,3.7,23.7,14.368,35.276,-11.560,1.72,3.56,2.000,-9.802,-137.668,-3.337,136.740000,2.127,150.110000,50.0,5.313,7.61,10.88,49.0,1.0,F,S,FF,5,0.0,2.015000e+09,0.0,2.0,0.0,3.0,0.0,0.0,0.0
3,-0.641,1.221,91.0,84.0,1289.590,169.751,-1.3,5.0,23.8,2.104,28.354,-20.540,1.74,3.35,2.000,-8.071,-133.005,-6.567,109.685636,2.279,187.463482,50.0,5.210,1.17,6.45,41.0,13.0,B,B,FF,6,0.0,2.015000e+09,0.0,2.0,0.0,4.0,0.0,0.0,0.0
4,-1.821,2.083,75.4,69.6,1374.569,280.671,18.4,12.0,23.8,-10.280,21.774,-34.111,1.72,3.56,2.000,-6.309,-110.409,0.325,146.527525,2.179,177.242829,50.0,5.557,-8.43,-1.65,18.0,13.0,B,B,CU,7,0.0,2.015000e+09,1.0,2.0,0.0,5.0,0.0,0.0,0.0
5,0.627,2.397,92.9,84.8,2743.856,148.110,-45.7,3.7,23.7,13.590,32.274,-10.333,1.72,3.56,2.000,-6.943,-136.012,-5.738,118.004772,2.273,164.467012,50.0,5.264,7.32,11.72,42.0,6.0,X,X,FF,8,0.0,2.015000e+09,2.0,2.0,0.0,6.0,0.0,0.0,0.0
6,-1.088,1.610,93.3,85.3,2848.535,147.044,-46.3,3.6,23.7,14.549,31.469,-9.734,1.59,3.45,2.000,-11.032,-136.208,-7.762,141.430000,2.013,205.810000,50.0,5.179,7.79,11.97,80.0,13.0,B,B,FF,12,0.0,2.015000e+09,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7,-0.257,2.047,89.3,82.4,1433.743,185.948,7.3,4.8,23.8,-1.339,27.421,-19.326,1.59,3.45,0.778,-6.335,-130.711,-4.611,186.410000,2.298,182.540000,50.0,5.284,-0.77,7.38,39.0,7.0,D,X,FC,13,0.0,2.015000e+09,1.0,0.0,1.0,2.0,0.0,0.0,0.0
8,1.470,2.350,92.1,85.0,2666.090,146.146,-45.0,4.0,23.8,13.808,28.169,-11.591,1.89,3.46,2.000,-5.075,-134.873,-5.723,93.100000,2.402,174.060000,50.0,5.310,7.46,11.09,42.0,14.0,B,B,FF,17,0.0,2.015000e+09,0.0,0.0,1.0,1.0,0.0,1.0,0.0
9,-1.337,1.898,89.3,82.0,1384.143,174.388,2.0,4.9,23.8,1.218,28.828,-19.782,1.81,3.52,0.648,-9.239,-130.512,-4.904,135.831493,2.165,182.991946,50.0,5.302,0.71,7.18,34.0,13.0,B,B,FF,18,0.0,2.015000e+09,1.0,0.0,1.0,2.0,0.0,1.0,0.0


In [4]:
pitches.describe()

,px,pz,start_speed,end_speed,spin_rate,spin_dir,break_angle,break_length,break_y,ax,ay,az,sz_bot,sz_top,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,pfx_x,pfx_z,nasty,zone,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
count,2.852965e+06,2.852965e+06,2.853040e+06,2.853040e+06,2.852965e+06,2.852965e+06,2.852965e+06,2.852965e+06,2.852965e+06,2.852965e+06,2.852965e+06,2.852965e+06,2.865071e+06,2.865071e+06,2.852965e+06,2.852965e+06,2.852965e+06,2.852965e+06,2.867154e+06,2.852965e+06,2.867154e+06,2.852965e+06,2.852965e+06,2.853012e+06,2.853012e+06,2.852965e+06,2.852965e+06,2.867154e+06,2.867154e+06,2.867154e+06,2.867154e+06,2.867154e+06,2.867154e+06,2.867154e+06,2.867154e+06,2.867154e+06,2.867154e+06
mean,6.572502e-03,2.254962e+00,8.838124e+01,8.136274e+01,1.731173e+03,1.802308e+02,5.848084e+00,6.601459e+00,2.381685e+01,-2.308512e+00,2.649175e+01,-2.278275e+01,1.566689e+00,3.435719e+00,1.550896e+00,2.282811e+00,-1.288567e+02,-4.420795e+00,1.159975e+02,-7.189111e-01,1.758503e+02,5.000000e+01,5.814139e+00,-1.139971e+00,5.046326e+00,4.442669e+01,9.819750e+00,3.129711e+02,2.256608e+00,2.016606e+09,8.824259e-01,8.810873e-01,9.826033e-01,2.894472e+00,3.018314e-01,1.860451e-01,9.595543e-02
std,8.927490e-01,9.463968e-01,6.015540e+00,5.364057e+00,6.827521e+02,6.742859e+01,2.443621e+01,1.331708e+02,6.446540e-02,1.074066e+01,4.142729e+00,8.865066e+00,1.549407e-01,2.172590e-01,5.537377e-01,5.990039e+00,8.726126e+00,3.067389e+00,3.409448e+01,1.728592e+00,2.960411e+01,3.120278e-14,4.578852e-01,6.275731e+00,5.187005e+00,1.683662e+01,4.060956e+00,1.928482e+02,2.550955e+00,1.117238e+06,9.665457e-01,8.251786e-01,8.174947e-01,1.726595e+00,4.590526e-01,3.891431e-01,2.945302e-01
min,-1.054333e+01,-5.183664e+00,3.390000e+01,3.240000e+01,1.214000e+00,-2.000000e-03,-9.000000e+01,1.000000e-01,2.330000e+01,-5.929009e+01,-5.243748e-01,-7.717142e+01,-7.921409e+00,-1.849129e+00,0.000000e+00,-2.486900e+01,-1.533620e+02,-1.977060e+01,0.000000e+00,-9.556000e+00,0.000000e+00,5.000000e+01,-1.941000e+00,-2.809165e+01,-7.020600e+01,0.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,2.015000e+09,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,-5.908243e-01,1.650000e+00,8.430000e+01,7.790000e+01,1.238745e+03,1.371540e+02,-1.190000e+01,4.300000e+00,2.380000e+01,-1.129000e+01,2.346000e+01,-2.871775e+01,1.485351e+00,3.300000e+00,9.060000e-01,-2.081861e+00,-1.355100e+02,-6.513000e+00,9.342000e+01,-1.955000e+00,1.606500e+02,5.000000e+01,5.564194e+00,-6.450000e+00,2.160000e+00,3.200000e+01,6.000000e+00,1.490000e+02,0.000000e+00,2.016004e+09,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.300000e-02,2.259000e+00,8.970000e+01,8.250000e+01,1.866322e+03,1.958850e+02,6.800000e+00,6.000000e+00,2.380000e+01,-3.142000e+00,2.647444e+01,-2.159500e+01,1.570000e+00,3.440000e+00,2.000000e+00,3.989000e+00,-1.308299e+02,-4.604841e+00,1.161400e+02,-1.333530e+00,1.771800e+02,5.000000e+01,5.842237e+00,-1.880000e+00,6.090000e+00,4.400000e+01,1.100000e+01,3.020000e+02,1.000000e+00,2.017004e+09,1.000000e+00,1.000000e+00,1.000000e+00,3.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
75%,6.090000e-01,2.863000e+00,9.300000e+01,8.540000e+01,2.247564e+03,2.252340e+02,2.700000e+01,8.000000e+00,2.390000e+01,5.995827e+00,2.938409e+01,-1.574300e+01,1.658626e+00,3.578467e+00,2.000000e+00,6.625942e+00,-1.229450e+02,-2.488206e+00,1.392400e+02,7.766212e-01,1.936400e+02,5.000000e+01,6.104551e+00,3.864644e+00,8.950000e+00,5.600000e+01,1.300000e+01,4.640000e+02,3.000000e+00,2.018002e+09,2.000000e+00,2.000000e+00,2.000000e+00,4.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
max,1.295291e+01,1.488624e+01,1.050000e+02,9.690000e+01,6.539259e+03,3.600010e+02,2.694000e+02,2.248893e+05,3.640000e+01,4.097800e+01,5.405700e+01,2.230520e+01,1.069854e+01,1.054815e+01,2.000000e+00,2.515000e+01,-4.755200e+01,2.781500e+01,2.499500e+02,9.676000e+00,2.500000e+02,5.000000e+01,1.059600e+01,2.202000e+01,3.31

In [5]:
pitches.type_confidence.describe()

count    2.852965e+06
mean     1.550896e+00
std      5.537377e-01
min      0.000000e+00
25%      9.060000e-01
50%      2.000000e+00
75%      2.000000e+00
max      2.000000e+00
Name: type_confidence, dtype: float64

In [6]:
pitches.pitch_type.unique()

array(['FF', 'CU', 'FC', 'SI', 'CH', 'FT', 'IN', 'SL', nan, 'KC', 'EP',
       'FS', 'FO', 'PO', 'KN', 'UN', 'SC', 'FA', 'AB'], dtype=object)

### Observation
(Maybe we can create a model to predict what pitches these lower `type_confidence` values are!)
- It looks like most of the ``confidence_type`` values are ``2.000`` while others are less than this. Let's assume that ``2.000`` is maximum confidence
- It's hard to determine what a lot of these values are, but many seem to have to do with the mechanical characteristics of the pitches (i.e., speed and position of the pitches, spin and break, etc.). These are important! Others seem to have to do with strike zone, *nastiness*, game state (e.g., ``b_score``, ``b_count``, ``s_count``, etc). These are probably not so important!
- We may want join this data to the ``at_bat`` and ``pitcher`` data, so we probably want to hang on to the ``ab_id``

In [7]:
# # drop unwanted columns
# unwanted_columns = [
#     'sz_bot',
#     'sz_top',
#     'nasty'
# ]
# pitches.drop(columns=unwanted_columns, inplace=True)
# pitches.head(10)

In [8]:
# clean up the data a bit
pitches['zone'].fillna(-1, inplace=True)  # for now, assign -1 for all empty zone values
pitches['type_confidence'].fillna(-1, inplace=True) # assuming that type confidence ranges from 0 to 2, assign 0 for all empty type_confidence values
pitches['pitch_type'].fillna('UNK', inplace=True) # based on the pitch type definitions, assign 'UN' for all empty pitch_type values
# pitches = pitches.astype({
#     'zone':'int32',
#     'b_score':'int32',
#     'ab_id':'int32',
#     'b_count':'int32',
#     's_count':'int32',
#     'outs':'int32',
#     'pitch_num':'int32',
#     'on_1b':'int32',
#     'on_2b':'int32',
#     'on_3b':'int32'
# })
pitches.head(10)

,px,pz,start_speed,end_speed,spin_rate,spin_dir,break_angle,break_length,break_y,ax,ay,az,sz_bot,sz_top,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,pfx_x,pfx_z,nasty,zone,code,type,pitch_type,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b
0,0.416,2.963,92.9,84.1,2305.052,159.235,-25.0,3.2,23.7,7.665,34.685,-11.960,1.72,3.56,2.000,-6.409,-136.065,-3.995,101.140000,2.280,158.780000,50.0,5.302,4.16,10.93,55.0,3.0,C,S,FF,3,0.0,2.015000e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,-0.191,2.347,92.8,84.1,2689.935,151.402,-40.7,3.4,23.7,12.043,34.225,-10.085,1.72,3.56,2.000,-8.411,-135.690,-5.980,124.280000,2.119,175.410000,50.0,5.307,6.57,12.00,31.0,5.0,S,S,FF,4,0.0,2.015000e+09,0.0,1.0,0.0,2.0,0.0,0.0,0.0
2,-0.518,3.284,94.1,85.2,2647.972,145.125,-43.7,3.7,23.7,14.368,35.276,-11.560,1.72,3.56,2.000,-9.802,-137.668,-3.337,136.740000,2.127,150.110000,50.0,5.313,7.61,10.88,49.0,1.0,F,S,FF,5,0.0,2.015000e+09,0.0,2.0,0.0,3.0,0.0,0.0,0.0
3,-0.641,1.221,91.0,84.0,1289.590,169.751,-1.3,5.0,23.8,2.104,28.354,-20.540,1.74,3.35,2.000,-8.071,-133.005,-6.567,109.685636,2.279,187.463482,50.0,5.210,1.17,6.45,41.0,13.0,B,B,FF,6,0.0,2.015000e+09,0.0,2.0,0.0,4.0,0.0,0.0,0.0
4,-1.821,2.083,75.4,69.6,1374.569,280.671,18.4,12.0,23.8,-10.280,21.774,-34.111,1.72,3.56,2.000,-6.309,-110.409,0.325,146.527525,2.179,177.242829,50.0,5.557,-8.43,-1.65,18.0,13.0,B,B,CU,7,0.0,2.015000e+09,1.0,2.0,0.0,5.0,0.0,0.0,0.0
5,0.627,2.397,92.9,84.8,2743.856,148.110,-45.7,3.7,23.7,13.590,32.274,-10.333,1.72,3.56,2.000,-6.943,-136.012,-5.738,118.004772,2.273,164.467012,50.0,5.264,7.32,11.72,42.0,6.0,X,X,FF,8,0.0,2.015000e+09,2.0,2.0,0.0,6.0,0.0,0.0,0.0
6,-1.088,1.610,93.3,85.3,2848.535,147.044,-46.3,3.6,23.7,14.549,31.469,-9.734,1.59,3.45,2.000,-11.032,-136.208,-7.762,141.430000,2.013,205.810000,50.0,5.179,7.79,11.97,80.0,13.0,B,B,FF,12,0.0,2.015000e+09,0.0,0.0,1.0,1.0,0.0,0.0,0.0
7,-0.257,2.047,89.3,82.4,1433.743,185.948,7.3,4.8,23.8,-1.339,27.421,-19.326,1.59,3.45,0.778,-6.335,-130.711,-4.611,186.410000,2.298,182.540000,50.0,5.284,-0.77,7.38,39.0,7.0,D,X,FC,13,0.0,2.015000e+09,1.0,0.0,1.0,2.0,0.0,0.0,0.0
8,1.470,2.350,92.1,85.0,2666.090,146.146,-45.0,4.0,23.8,13.808,28.169,-11.591,1.89,3.46,2.000,-5.075,-134.873,-5.723,93.100000,2.402,174.060000,50.0,5.310,7.46,11.09,42.0,14.0,B,B,FF,17,0.0,2.015000e+09,0.0,0.0,1.0,1.0,0.0,1.0,0.0
9,-1.337,1.898,89.3,82.0,1384.143,174.388,2.0,4.9,23.8,1.218,28.828,-19.782,1.81,3.52,0.648,-9.239,-130.512,-4.904,135.831493,2.165,182.991946,50.0,5.302,0.71,7.18,34.0,13.0,B,B,FF,18,0.0,2.015000e+09,1.0,0.0,1.0,2.0,0.0,1.0,0.0


In [9]:
at_bats = pd.read_csv('atbats.csv') 
at_bats.head(10)

# drop unwanted columns
unwanted_columns = ['event']
at_bats.drop(columns=unwanted_columns, inplace=True)
at_bats.head(10)

,ab_id,batter_id,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
0,2015000001,572761,201500001,1,1,0,L,452657,L,True
1,2015000002,518792,201500001,1,1,0,L,452657,L,True
2,2015000003,407812,201500001,1,1,0,L,452657,R,True
3,2015000004,425509,201500001,1,2,0,L,452657,R,True
4,2015000005,571431,201500001,1,3,0,L,452657,L,True
5,2015000006,451594,201500001,1,0,1,R,425794,L,False
6,2015000007,624585,201500001,1,1,1,R,425794,R,False
7,2015000008,519203,201500001,1,2,1,R,425794,L,False
8,2015000009,516770,201500001,1,3,1,R,425794,R,False
9,2015000010,425877,201500001,2,1,0,L,452657,R,True


## Merge dataframes and do final cleanup

In [10]:
mlb_df = pd.merge(left=pitches, right=at_bats, on=['ab_id'], how='left')
mlb_df.head(10)

,px,pz,start_speed,end_speed,spin_rate,spin_dir,break_angle,break_length,break_y,ax,ay,az,sz_bot,sz_top,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,pfx_x,pfx_z,nasty,zone,code,type,pitch_type,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,batter_id,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
0,0.416,2.963,92.9,84.1,2305.052,159.235,-25.0,3.2,23.7,7.665,34.685,-11.960,1.72,3.56,2.000,-6.409,-136.065,-3.995,101.140000,2.280,158.780000,50.0,5.302,4.16,10.93,55.0,3.0,C,S,FF,3,0.0,2.015000e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
1,-0.191,2.347,92.8,84.1,2689.935,151.402,-40.7,3.4,23.7,12.043,34.225,-10.085,1.72,3.56,2.000,-8.411,-135.690,-5.980,124.280000,2.119,175.410000,50.0,5.307,6.57,12.00,31.0,5.0,S,S,FF,4,0.0,2.015000e+09,0.0,1.0,0.0,2.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
2,-0.518,3.284,94.1,85.2,2647.972,145.125,-43.7,3.7,23.7,14.368,35.276,-11.560,1.72,3.56,2.000,-9.802,-137.668,-3.337,136.740000,2.127,150.110000,50.0,5.313,7.61,10.88,49.0,1.0,F,S,FF,5,0.0,2.015000e+09,0.0,2.0,0.0,3.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
3,-0.641,1.221,91.0,84.0,1289.590,169.751,-1.3,5.0,23.8,2.104,28.354,-20.540,1.74,3.35,2.000,-8.071,-133.005,-6.567,109.685636,2.279,187.463482,50.0,5.210,1.17,6.45,41.0,13.0,B,B,FF,6,0.0,2.015000e+09,0.0,2.0,0.0,4.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
4,-1.821,2.083,75.4,69.6,1374.569,280.671,18.4,12.0,23.8,-10.280,21.774,-34.111,1.72,3.56,2.000,-6.309,-110.409,0.325,146.527525,2.179,177.242829,50.0,5.557,-8.43,-1.65,18.0,13.0,B,B,CU,7,0.0,2.015000e+09,1.0,2.0,0.0,5.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
5,0.627,2.397,92.9,84.8,2743.856,148.110,-45.7,3.7,23.7,13.590,32.274,-10.333,1.72,3.56,2.000,-6.943,-136.012,-5.738,118.004772,2.273,164.467012,50.0,5.264,7.32,11.72,42.0,6.0,X,X,FF,8,0.0,2.015000e+09,2.0,2.0,0.0,6.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
6,-1.088,1.610,93.3,85.3,2848.535,147.044,-46.3,3.6,23.7,14.549,31.469,-9.734,1.59,3.45,2.000,-11.032,-136.208,-7.762,141.430000,2.013,205.810000,50.0,5.179,7.79,11.97,80.0,13.0,B,B,FF,12,0.0,2.015000e+09,0.0,0.0,1.0,1.0,0.0,0.0,0.0,518792,201500001,1,1,0,L,452657,L,True
7,-0.257,2.047,89.3,82.4,1433.743,185.948,7.3,4.8,23.8,-1.339,27.421,-19.326,1.59,3.45,0.778,-6.335,-130.711,-4.611,186.410000,2.298,182.540000,50.0,5.284,-0.77,7.38,39.0,7.0,D,X,FC,13,0.0,2.015000e+09,1.0,0.0,1.0,2.0,0.0,0.0,0.0,518792,201500001,1,1,0,L,452657,L,True
8,1.470,2.350,92.1,85.0,2666.090,146.146,-45.0,4.0,23.8,13.808,28.169,-11.591,1.89,3.46,2.000,-5.075,-134.873,-5.723,93.100000,2.402,174.060000,50.0,5.310,7.46,11.09,42.0,14.0,B,B,FF,17,0.0,2.015000e+09,0.0,0.0,1.0,1.0,0.0,1.0,0.0,407812,201500001,1,1,0,L,452657,R,True
9,-1.337,1.898,89.3,82.0,1384.143,174.388,2.0,4.9,23.8,1.218,28.828,-19.782,1.81,3.52,0.648,-9.239,-130.512,-4.904,135.831493,2.165,182.991946,50.0,5.302,0.71,7.18,34.0,13.0,B,B,FF,18,0.0,2.015000e+09,1.0,0.0,1.0,2.0,0.0,1.0,0.0,407812,201500001,1,1,0,L,452657,R,True


In [11]:
# reset the index
mlb_df = mlb_df.reset_index()
mlb_df.drop(columns=['index'], inplace=True)
mlb_df.head(10)

,px,pz,start_speed,end_speed,spin_rate,spin_dir,break_angle,break_length,break_y,ax,ay,az,sz_bot,sz_top,type_confidence,vx0,vy0,vz0,x,x0,y,y0,z0,pfx_x,pfx_z,nasty,zone,code,type,pitch_type,event_num,b_score,ab_id,b_count,s_count,outs,pitch_num,on_1b,on_2b,on_3b,batter_id,g_id,inning,o,p_score,p_throws,pitcher_id,stand,top
0,0.416,2.963,92.9,84.1,2305.052,159.235,-25.0,3.2,23.7,7.665,34.685,-11.960,1.72,3.56,2.000,-6.409,-136.065,-3.995,101.140000,2.280,158.780000,50.0,5.302,4.16,10.93,55.0,3.0,C,S,FF,3,0.0,2.015000e+09,0.0,0.0,0.0,1.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
1,-0.191,2.347,92.8,84.1,2689.935,151.402,-40.7,3.4,23.7,12.043,34.225,-10.085,1.72,3.56,2.000,-8.411,-135.690,-5.980,124.280000,2.119,175.410000,50.0,5.307,6.57,12.00,31.0,5.0,S,S,FF,4,0.0,2.015000e+09,0.0,1.0,0.0,2.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
2,-0.518,3.284,94.1,85.2,2647.972,145.125,-43.7,3.7,23.7,14.368,35.276,-11.560,1.72,3.56,2.000,-9.802,-137.668,-3.337,136.740000,2.127,150.110000,50.0,5.313,7.61,10.88,49.0,1.0,F,S,FF,5,0.0,2.015000e+09,0.0,2.0,0.0,3.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
3,-0.641,1.221,91.0,84.0,1289.590,169.751,-1.3,5.0,23.8,2.104,28.354,-20.540,1.74,3.35,2.000,-8.071,-133.005,-6.567,109.685636,2.279,187.463482,50.0,5.210,1.17,6.45,41.0,13.0,B,B,FF,6,0.0,2.015000e+09,0.0,2.0,0.0,4.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
4,-1.821,2.083,75.4,69.6,1374.569,280.671,18.4,12.0,23.8,-10.280,21.774,-34.111,1.72,3.56,2.000,-6.309,-110.409,0.325,146.527525,2.179,177.242829,50.0,5.557,-8.43,-1.65,18.0,13.0,B,B,CU,7,0.0,2.015000e+09,1.0,2.0,0.0,5.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
5,0.627,2.397,92.9,84.8,2743.856,148.110,-45.7,3.7,23.7,13.590,32.274,-10.333,1.72,3.56,2.000,-6.943,-136.012,-5.738,118.004772,2.273,164.467012,50.0,5.264,7.32,11.72,42.0,6.0,X,X,FF,8,0.0,2.015000e+09,2.0,2.0,0.0,6.0,0.0,0.0,0.0,572761,201500001,1,1,0,L,452657,L,True
6,-1.088,1.610,93.3,85.3,2848.535,147.044,-46.3,3.6,23.7,14.549,31.469,-9.734,1.59,3.45,2.000,-11.032,-136.208,-7.762,141.430000,2.013,205.810000,50.0,5.179,7.79,11.97,80.0,13.0,B,B,FF,12,0.0,2.015000e+09,0.0,0.0,1.0,1.0,0.0,0.0,0.0,518792,201500001,1,1,0,L,452657,L,True
7,-0.257,2.047,89.3,82.4,1433.743,185.948,7.3,4.8,23.8,-1.339,27.421,-19.326,1.59,3.45,0.778,-6.335,-130.711,-4.611,186.410000,2.298,182.540000,50.0,5.284,-0.77,7.38,39.0,7.0,D,X,FC,13,0.0,2.015000e+09,1.0,0.0,1.0,2.0,0.0,0.0,0.0,518792,201500001,1,1,0,L,452657,L,True
8,1.470,2.350,92.1,85.0,2666.090,146.146,-45.0,4.0,23.8,13.808,28.169,-11.591,1.89,3.46,2.000,-5.075,-134.873,-5.723,93.100000,2.402,174.060000,50.0,5.310,7.46,11.09,42.0,14.0,B,B,FF,17,0.0,2.015000e+09,0.0,0.0,1.0,1.0,0.0,1.0,0.0,407812,201500001,1,1,0,L,452657,R,True
9,-1.337,1.898,89.3,82.0,1384.143,174.388,2.0,4.9,23.8,1.218,28.828,-19.782,1.81,3.52,0.648,-9.239,-130.512,-4.904,135.831493,2.165,182.991946,50.0,5.302,0.71,7.18,34.0,13.0,B,B,FF,18,0.0,2.015000e+09,1.0,0.0,1.0,2.0,0.0,1.0,0.0,407812,201500001,1,1,0,L,452657,R,True


In [12]:
low_confidence_df = mlb_df[mlb_df.type_confidence < 2.000]
print(f"low confidence count: {len(low_confidence_df)}")
print(f"low confidence/unknown pitch_type count: {len(low_confidence_df[low_confidence_df.pitch_type == 'UN'])}")

low confidence count: 1155992
low confidence/unknown pitch_type count: 57


In [13]:
full_confidence_df = mlb_df[mlb_df.type_confidence >= 2.000]
print(f"full confidence count: {len(full_confidence_df)}")
print(f"full confidence/unknown pitch_type count: {len(full_confidence_df[full_confidence_df.pitch_type == 'UN'])}")

full confidence count: 1711162
full confidence/unknown pitch_type count: 0


### First model attempt
For a first pass, let's use what I would consider the strongest signals/features: ``start_speed``, ``end_speed``, ``spin_rate``, ``spin_dir``, ``break_angle``, and ``break_length``.

In [14]:
y = pd.get_dummies(full_confidence_df.pitch_type)
print(y.columns)
y.head(10)

Index(['CH', 'CU', 'EP', 'FA', 'FC', 'FF', 'FO', 'FS', 'FT', 'KC', 'KN', 'SC',
       'SI', 'SL'],
      dtype='object')


,CH,CU,EP,FA,FC,FF,FO,FS,FT,KC,KN,SC,SI,SL
0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0,0,0,0,0
6,0,0,0,0,0,1,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,0,0,0,0,0
10,0,0,0,0,0,1,0,0,0,0,0,0,0,0
11,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [15]:
features = ['start_speed', 'end_speed', 'spin_rate', 'spin_dir', 'break_angle', 'break_length']
X = full_confidence_df[features]

In [16]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
from sklearn.tree import DecisionTreeClassifier

# Define model. Specify a number for random_state to ensure same results each run
pitch_predictor_model = DecisionTreeClassifier(
    random_state = 42, 
    max_depth = 5
)

# Fit model
pitch_predictor_model.fit(X_train, y_train)

DecisionTreeClassifier(max_depth=5, random_state=42)

In [18]:
y_pred = pitch_predictor_model.predict(X_test)

In [19]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)

ValueError: multilabel-indicator is not supported

In [ ]:
print(y.columns)
i = 4
print(y_pred[i])
print(y_test.iloc[i])

In [ ]:
from sklearn.metrics import log_loss

log_loss(y_test, y_pred)

In [ ]:
# p = pd.DataFrame({"monkey":[0,1,0,0,0],"rabbit":[1,0,0,0,0],"fox":[0,0,1,0,0]})
# ['CH', 'CU', 'EP', 'FA','FC', 'FF', 'FO', 'FS','FT', 'KC', 'KN', 'SC','SI', 'SL']

y.columns

In [ ]:
def get_pitch_type(row):
    for i in range(len(y.columns)):
        if row[i] == 1:
            return y.columns[i]

In [ ]:
import numpy as np
y_pred_pitch_type = np.apply_along_axis(get_pitch_type, 1, y_pred)

In [ ]:
y_pred_pitch_type[:10]

In [ ]:
changeups = full_confidence_df[full_confidence_df.pitch_type == 'CH']
changeups.head(10)